In [11]:
import requests
import pandas as pd
import json
from io import StringIO
import psycopg2



url = "https://odds.p.rapidapi.com/v4/sports/soccer_brazil_campeonato/scores"

querystring = {"daysFrom": "3"}

headers = {
	"X-RapidAPI-Key": "56ef5d7821mshb3a5e1905772197p1f12d6jsn880239b1ac0a",
	"X-RapidAPI-Host": "odds.p.rapidapi.com"
}

response = requests.get(url, headers=headers, params=querystring)
dados = response.json()

# normalizando Json e criando um dataFrame
cria_df = pd.json_normalize(dados)
df_main = pd.DataFrame(cria_df)

df_main.head(10)
#jogos.dtypes

,id,sport_key,sport_title,commence_time,completed,home_team,away_team,scores,last_update
0,a7a6974300929a2b7bcee4dfceb2ad71,soccer_brazil_campeonato,Brazil Série A,2023-10-08T19:00:02Z,True,Internacional,Gremio,"[{'name': 'Internacional', 'score': '3'}, {'na...",2023-10-09T05:01:04Z
1,de74ae1f97356debe22007847a74ad9d,soccer_brazil_campeonato,Brazil Série A,2023-10-08T19:00:30Z,True,Fluminense,Botafogo,"[{'name': 'Fluminense', 'score': '0'}, {'name'...",2023-10-09T05:01:04Z
2,72da1530e9470614e8b34f858921428a,soccer_brazil_campeonato,Brazil Série A,2023-10-08T19:01:49Z,True,Palmeiras,Santos,"[{'name': 'Palmeiras', 'score': '1'}, {'name':...",2023-10-09T05:01:04Z
3,f4a023dede2cc766ad8c289195ed02b2,soccer_brazil_campeonato,Brazil Série A,2023-10-08T21:30:10Z,True,Atletico Mineiro,Coritiba,"[{'name': 'Atletico Mineiro', 'score': '1'}, {...",2023-10-09T05:01:04Z
4,784db10ad32e4b6e639ade38203f854d,soccer_brazil_campeonato,Brazil Série A,2023-10-08T21:30:21Z,True,Fortaleza,América Mineiro,"[{'name': 'Fortaleza', 'score': '3'}, {'name':...",2023-10-09T05:01:04Z
5,2eac87ab308b20e721a29f2353445107,soccer_brazil_campeonato,Brazil Série A,2023-10-08T21:30:25Z,True,Atletico Paranaense,Bragantino-SP,"[{'name': 'Atletico Paranaense', 'score': '1'}...",2023-10-09T05:01:04Z
6,276ea760baf0fc0bbfca22d2bff7285a,soccer_brazil_campeonato,Brazil Série A,2023-10-15T00:00:00Z,False,Cuiabá,Cruzeiro,None,None
7,d9e643a2510f34a219fd5904648a4275,soccer_brazil_campeonato,Brazil Série A,2023-10-18T00:30:00Z,False,Goiás,Sao Paulo,None,None
8,75456c31a9d7fec02978ce8b83027127,soccer_brazil_campeonato,Brazil Série A,2023-10-18T22:00:00Z,False,Gremio,Atletico Paranaense,None,None
9,307d881e844eae842ece62b10e02a5ef,soccer_brazil_campeonato,Brazil Série A,2023-10-18T23:00:00Z,False,América Mineiro,Botafogo,None,None


In [12]:
# separando a data e hora das partidas
data = df_main['commence_time']
df_main['commence_time'] = pd.to_datetime(df_main['commence_time'])

df_main['data-partida'] = df_main['commence_time'].dt.date
df_main['hora-partida'] = df_main['commence_time'].dt.time

# Exiba as primeiras linhas do DataFrame com as colunas separadas
print(df_main[['data-partida', 'hora-partida']].head())
   

  data-partida hora-partida
0   2023-10-08     19:00:02
1   2023-10-08     19:00:30
2   2023-10-08     19:01:49
3   2023-10-08     21:30:10
4   2023-10-08     21:30:21


In [13]:
df_main.head()

,id,sport_key,sport_title,commence_time,completed,home_team,away_team,scores,last_update,data-partida,hora-partida
0,a7a6974300929a2b7bcee4dfceb2ad71,soccer_brazil_campeonato,Brazil Série A,2023-10-08 19:00:02+00:00,True,Internacional,Gremio,"[{'name': 'Internacional', 'score': '3'}, {'na...",2023-10-09T05:01:04Z,2023-10-08,19:00:02
1,de74ae1f97356debe22007847a74ad9d,soccer_brazil_campeonato,Brazil Série A,2023-10-08 19:00:30+00:00,True,Fluminense,Botafogo,"[{'name': 'Fluminense', 'score': '0'}, {'name'...",2023-10-09T05:01:04Z,2023-10-08,19:00:30
2,72da1530e9470614e8b34f858921428a,soccer_brazil_campeonato,Brazil Série A,2023-10-08 19:01:49+00:00,True,Palmeiras,Santos,"[{'name': 'Palmeiras', 'score': '1'}, {'name':...",2023-10-09T05:01:04Z,2023-10-08,19:01:49
3,f4a023dede2cc766ad8c289195ed02b2,soccer_brazil_campeonato,Brazil Série A,2023-10-08 21:30:10+00:00,True,Atletico Mineiro,Coritiba,"[{'name': 'Atletico Mineiro', 'score': '1'}, {...",2023-10-09T05:01:04Z,2023-10-08,21:30:10
4,784db10ad32e4b6e639ade38203f854d,soccer_brazil_campeonato,Brazil Série A,2023-10-08 21:30:21+00:00,True,Fortaleza,América Mineiro,"[{'name': 'Fortaleza', 'score': '3'}, {'name':...",2023-10-09T05:01:04Z,2023-10-08,21:30:21


In [14]:
# seleciona colunas 
jogos_new = df_main[['data-partida', 'hora-partida', 'home_team', 'away_team', 'scores']]
jogos_new.head()

,data-partida,hora-partida,home_team,away_team,scores
0,2023-10-08,19:00:02,Internacional,Gremio,"[{'name': 'Internacional', 'score': '3'}, {'na..."
1,2023-10-08,19:00:30,Fluminense,Botafogo,"[{'name': 'Fluminense', 'score': '0'}, {'name'..."
2,2023-10-08,19:01:49,Palmeiras,Santos,"[{'name': 'Palmeiras', 'score': '1'}, {'name':..."
3,2023-10-08,21:30:10,Atletico Mineiro,Coritiba,"[{'name': 'Atletico Mineiro', 'score': '1'}, {..."
4,2023-10-08,21:30:21,Fortaleza,América Mineiro,"[{'name': 'Fortaleza', 'score': '3'}, {'name':..."


In [15]:
scorejson = jogos_new['scores']
json_str = scorejson.to_json(orient='records')
json_obj = json.loads(json_str)

normal_json = json.dumps(json_obj, indent=1)

print(normal_json)

[
 [
  {
   "name": "Internacional",
   "score": "3"
  },
  {
   "name": "Gremio",
   "score": "2"
  }
 ],
 [
  {
   "name": "Fluminense",
   "score": "0"
  },
  {
   "name": "Botafogo",
   "score": "2"
  }
 ],
 [
  {
   "name": "Palmeiras",
   "score": "1"
  },
  {
   "name": "Santos",
   "score": "2"
  }
 ],
 [
  {
   "name": "Atletico Mineiro",
   "score": "1"
  },
  {
   "name": "Coritiba",
   "score": "2"
  }
 ],
 [
  {
   "name": "Fortaleza",
   "score": "3"
  },
  {
   "name": "Am\u00e9rica Mineiro",
   "score": "2"
  }
 ],
 [
  {
   "name": "Atletico Paranaense",
   "score": "1"
  },
  {
   "name": "Bragantino-SP",
   "score": "1"
  }
 ],
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null,
 null
]


In [23]:
# usando Lista com os dados manual :( 
data = [
    [
        "3",
        "2"
    ],
    [
        "0",
        "2"
    ],
    [
        "1",
        "2"
    ],
    [
        "1",
        "2"
    ],
    [
        "3",
        "2"
    ],
    [
        "1",
        "1"
    ]
]

# Criar um DataFrame a partir da lista
df_fim = pd.DataFrame(data)
df_fim.columns = [f"gols time {i}" for i in range(1, len(df_fim.columns) + 1)]



df_fim.head(10)

,gols time 1,gols time 2
0,3,2
1,0,2
2,1,2
3,1,2
4,3,2
5,1,1


In [24]:
df_final = pd.concat([jogos_new, df_fim], axis=1)
df_final.head(10)

,data-partida,hora-partida,home_team,away_team,scores,gols time 1,gols time 2
0,2023-10-08,19:00:02,Internacional,Gremio,"[{'name': 'Internacional', 'score': '3'}, {'na...",3,2
1,2023-10-08,19:00:30,Fluminense,Botafogo,"[{'name': 'Fluminense', 'score': '0'}, {'name'...",0,2
2,2023-10-08,19:01:49,Palmeiras,Santos,"[{'name': 'Palmeiras', 'score': '1'}, {'name':...",1,2
3,2023-10-08,21:30:10,Atletico Mineiro,Coritiba,"[{'name': 'Atletico Mineiro', 'score': '1'}, {...",1,2
4,2023-10-08,21:30:21,Fortaleza,América Mineiro,"[{'name': 'Fortaleza', 'score': '3'}, {'name':...",3,2
5,2023-10-08,21:30:25,Atletico Paranaense,Bragantino-SP,"[{'name': 'Atletico Paranaense', 'score': '1'}...",1,1
6,2023-10-15,00:00:00,Cuiabá,Cruzeiro,None,NaN,NaN
7,2023-10-18,00:30:00,Goiás,Sao Paulo,None,NaN,NaN
8,2023-10-18,22:00:00,Gremio,Atletico Paranaense,None,NaN,NaN
9,2023-10-18,23:00:00,América Mineiro,Botafogo,None,NaN,NaN


In [29]:
jogos = df_final.drop(columns=['scores'])

jogos.rename(columns={"data-partida": "data_jogo"}, inplace=True)
jogos.rename(columns={"hora-partida": "hora_jogo"}, inplace=True)
jogos.rename(columns={"home_team": "time_casa"}, inplace=True)
jogos.rename(columns={"away_team": "time_fora"}, inplace=True)

jogos.head()

,data_jogo,hora_jogo,time_casa,time_fora,gols time 1,gols time 2
0,2023-10-08,19:00:02,Internacional,Gremio,3,2
1,2023-10-08,19:00:30,Fluminense,Botafogo,0,2
2,2023-10-08,19:01:49,Palmeiras,Santos,1,2
3,2023-10-08,21:30:10,Atletico Mineiro,Coritiba,1,2
4,2023-10-08,21:30:21,Fortaleza,América Mineiro,3,2


In [ ]:

def conectar_banco():
    try:
        conn = psycopg2.connect(
            host="localhost",
            port="5433",
            database="prd_case",
            user="postgres",
            password="@Chupa123"
        )
        return conn
    except (Exception, psycopg2.Error) as error:
        print("Erro ao conectar ao PostgreSQL:", error)
        return None



conexao = conectar_banco()

if conexao:
    insert_jogos = """INSERT INTO jogos (
        data_jogo,
        hora_jogo,
        time_casa,
        time_fora,
        gols_time1,
        gols_time2
    ) VALUES (%s, %s, %s, %s, %s, %s)"""

    for i, row in jogos.iterrows():
        cursor = conexao.cursor()
        cursor.execute(insert_jogos, tuple(row))
        conexao.commit()
        cursor.close()
